In [2]:
from operator import attrgetter
from copy import deepcopy
from time import time
import random

In [3]:
def find_tile_position(number, n):
    if number != '-': 
        number = int(number)
    else: 
        number = n*n

    row = (number - 1) // n
    col = (number - 1) % n

    return row, col

In [4]:
class State:
    def __init__(self, depth, parent):
        self.depth = depth
        self.parent = parent
        self.board = []
        self.fscore = 0
        self.hscore = 0
        self.blank = (0, 0)

    def setBoard(self, board):
        self.board = deepcopy(board)
        return self

    def find(self, val):
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                if self.board[i][j] == val:
                    return i,j

    def setFscore(self, n):
        self.hscore = 0
        for x in range(len(self.board)):
            for y in range(len(self.board)):
                hx,hy = find_tile_position(self.board[x][y], n)
                self.hscore += abs(x - hx) + abs(y - hy)
        self.fscore = self.hscore + self.depth
        return self


In [5]:
# Create a randomized board for solving
def randBoard(rb, n):
    # attempt 10,000 shuffles
    x,y = n-1,n-1
    for _ in range(10000):
        r = random.randint(0,3)
        if r == 0 and x+1 <= n-1:
            rb[x][y] = rb[x+1][y]
            rb[x+1][y] = '-'
            x += 1
        elif r == 1 and y+1 <= n-1:
            rb[x][y] = rb[x][y+1]
            rb[x][y+1] = '-'
            y +=1
        elif r == 2 and x-1 >= 0:
            rb[x][y] = rb[x-1][y]
            rb[x-1][y] = '-'
            x -= 1
        elif r == 3 and y-1 >= 0:
            rb[x][y] = rb[x][y-1]
            rb[x][y-1] = '-'
            y -= 1
    return rb

In [6]:
# Create a Child State
def createChild(state, x1, y1, x2, y2):
    newState = State(state.depth+1, state)
    newState.setBoard(state.board)
    newState.board[x1][y1] = newState.board[x2][y2]
    newState.board[x2][y2] = '-'
    newState.blank = (x2, y2)
    return newState

In [7]:
# Display Solution 
def show_solution(solution):
    print("Solution Found: ")
    for index,state in enumerate(solution):
        for row in state.board:
            for col in row:
                print(col, end=" ")
            print("")
        if index != len(solution)-1:
            print(" ||")
            print(" \\/")

In [8]:
SIZE = 3

# Goal State
# Initialize the board
goal = [[0]*SIZE for _ in range(SIZE)]
# Fill the board with numbers in order
count = 1
for i in range(SIZE):
    for j in range(SIZE):
        if count != SIZE*SIZE:
            goal[i][j] = f'{count}'
        else:
            goal[i][j] = '-'
        count += 1


In [9]:
avg_time = []
avg_len = []
for _ in range(30):
    # Solution list
    solution_3 = []

    # Open and Closed States lists
    open = []
    closed = []

    # Create Initial State
    init = randBoard(deepcopy(goal), SIZE)
    iState = State(0,0).setBoard(init).setFscore(SIZE)
    iState.blank = iState.find('-')
    open.append(iState)

    searching = True
    end = 0
    start = time()
    while(searching and len(open)!=0):
        
        # Find state with lowest Fscore
        open.sort(key=attrgetter('fscore'))
        current_state = open.pop(0)
        closed.append(current_state.board)

        # Stop Search and Get Result
        if current_state.board == goal:
            end = time()
            searching = False
            while(current_state != 0):
                solution_3.insert(0, current_state)
                current_state = current_state.parent
        
        # Generate Child States
        else:
            x,y = current_state.blank
            if x+1 <= SIZE-1:
                child = createChild(current_state, x, y, x+1, y)
                if child.board not in closed:
                    child.setFscore(SIZE)
                    open.append(child)

            if y+1 <= SIZE-1:
                child = createChild(current_state, x, y, x, y+1)
                if child.board not in closed:
                    child.setFscore(SIZE)
                    open.append(child)

            if x-1 >= 0:
                child = createChild(current_state, x, y, x-1, y)
                if child.board not in closed:
                    child.setFscore(SIZE)
                    open.append(child)

            if y-1 >= 0:
                child = createChild(current_state, x, y, x, y-1)
                if child.board not in closed:
                    child.setFscore(SIZE)
                    open.append(child)
    
    avg_time.append(end-start)
    avg_len.append(len(solution_3))

print(f"Average Search Time: {sum(avg_time)/len(avg_time)} \nAverage Solution Length: {sum(avg_len)/len(avg_len)}")

Average Search Time: 0.22407514254252117 
Average Solution Length: 23.333333333333332


In [38]:
SIZE = 4

# Goal State
# Initialize the board
goal = [[0]*SIZE for _ in range(SIZE)]
# Fill the board with numbers in order
count = 1
for i in range(SIZE):
    for j in range(SIZE):
        if count != SIZE*SIZE:
            goal[i][j] = f'{count}'
        else:
            goal[i][j] = '-'
        count += 1

# Solution list
solution_4 = []

# Open and Closed States lists
open_states = []
open_boards = []
closed = []

# Create Initial State
init = randBoard(deepcopy(goal), SIZE)
iState = State(0,0).setBoard(init).setFscore(SIZE)
iState.blank = iState.find('-')
open_states.append(iState)

In [39]:
searching = True
end = 0
start = time()
while(searching and len(open_states)!=0):
    
    # Find state with lowest Fscore
    open_states.sort(key=attrgetter('fscore'))
    current_state = open_states.pop(0)
    open_boards = list(map(lambda s: s.board, open_states))
    closed.append(current_state.board)

    # Stop Search and Get Result
    if current_state.board == goal:
        end = time()
        searching = False
        while(current_state != 0):
            solution_4.insert(0, current_state)
            current_state = current_state.parent
    
    # Generate Child States
    else:
        x,y = current_state.blank
        if x+1 <= SIZE-1:
            child = createChild(current_state, x, y, x+1, y)
            if child.board not in closed and child.board not in open_boards:
                child.setFscore(SIZE)
                open_states.append(child)
                open_boards.append(child.board)

        if y+1 <= SIZE-1:
            child = createChild(current_state, x, y, x, y+1)
            if child.board not in closed and child.board not in open_boards:
                child.setFscore(SIZE)
                open_states.append(child)
                open_boards.append(child.board)

        if x-1 >= 0:
            child = createChild(current_state, x, y, x-1, y)
            if child.board not in closed and child.board not in open_boards:
                child.setFscore(SIZE)
                open_states.append(child)
                open_boards.append(child.board)

        if y-1 >= 0:
            child = createChild(current_state, x, y, x, y-1)
            if child.board not in closed and child.board not in open_boards:
                child.setFscore(SIZE)
                open_states.append(child)
                open_boards.append(child.board)

if searching:
    print("No Solution Found")
else:
    time_4 = end - start
    print(f"Search Time: {time_4} \nSolution Length: {len(solution_4)}")
    show_solution(solution_4)